請完成 scikit-learn-practice 比賽(點擊連結可至競賽頁面)，讓大家熟悉 Scikit-learn 的比賽



總共有一千筆訓練資料、40個 features，二元分類問題，練習 features scaling、建模、調參數等步驟
每天最多上傳 10 次結果
請在 private / public leaderboard 上取得 0.7 以上的準確率
可多參考別人的 Kernel，學習別人的寫法與思路，完成自己的 Kaggle 競賽


作業提交請截圖kaggle競賽頁面提交畫面上傳至github，並回到官網提交github連結。(以下為Kaggle競賽頁面截圖範例)

In [2]:
from sklearn import datasets, metrics
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
import os
import numpy as np 
import pandas as pd

In [5]:
dir_path = './D48/'
train_fileName = os.path.join(dir_path, 'train.csv')
test_fileName = os.path.join(dir_path, 'test.csv')
trainLabels_fileName = os.path.join(dir_path, 'trainLabels.csv')

In [8]:
# 讀取檔案
train = pd.read_csv(train_fileName,header=None)
test = pd.read_csv(test_fileName,header=None)
trainLabels = pd.read_csv(trainLabels_fileName,header=None)
print(train.shape)
print(trainLabels.shape)

(1000, 40)
(1000, 1)


In [9]:
# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(train, trainLabels, test_size=0.2, random_state=4)
print('Training data shape: ', x_train.shape)
print('Testing data shape: ', x_test.shape)

Training data shape:  (800, 40)
Testing data shape:  (200, 40)


In [11]:
# 建立模型
clf_GBC = GradientBoostingClassifier()

# 訓練模型
clf_GBC.fit(x_train, y_train)

C:\Users\mouu\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [12]:
# 預測測試集
y_pred_GBC = clf_GBC.predict(x_test)
acc_GBC = metrics.accuracy_score(y_test, y_pred_GBC)
print("Acuuracy: ", acc_GBC)

Acuuracy:  0.86


In [13]:
# 設定要訓練的超參數組合
n_estimators = range(100,300,10)

max_depth = [3,5,7,9]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf_GBC, param_grid, scoring=metrics.make_scorer(metrics.accuracy_score), n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

C:\Users\mouu\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.


Fitting 3 folds for each of 80 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 188 tasks      | elapsed:   19.8s
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:   24.6s finished
C:\Users\mouu\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [14]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: 0.880000 using {'max_depth': 5, 'n_estimators': 190}


In [15]:
# 使用最佳參數重新建立模型
clf_bestparam = GradientBoostingClassifier(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

# 訓練模型
clf_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(x_test)

C:\Users\mouu\Anaconda3\lib\site-packages\sklearn\ensemble\gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [16]:
acc = metrics.accuracy_score(y_test, y_pred)
print("Acuuracy: ", acc)

Acuuracy:  0.885


In [17]:
clf_GBC_pred = clf_bestparam.predict(test)

In [18]:
# 計算提交結果
submit = pd.DataFrame(clf_GBC_pred)
submit.columns = ['Solution']
submit['Id'] = np.arange(1,submit.shape[0]+1)
submit = submit[['Id', 'Solution']]
submit.shape

(9000, 2)

In [19]:
submit.to_csv("Day48.csv",index=False)